In [ ]:
# Partie avancée
The model might focus on absolute price levels, which are often less informative than price movements.

#Valeur absolute de sous-jacent "KOSPI200", logiquement/financièrement, explique pas VKOSPI => Essayer d'effacer cette colonne là et le remplacer par les returns (%) d'un jour vs autre jour
# df["Return_KOSPI200"] = df["KOSPI200"].pct_change()
# print(df["Return_KOSPI200"])

# horizon = [2, 5, 30, 60]

# new_predictors = []
# for h in horizon:
#     rolling_mean = df["KOSPI200"].rolling(h).mean()
#     ratio_column = f"KOSPI200_Ratio_{h}"
#     df[ratio_column] = df["KOSPI200"] / rolling_mean

# df.head()


# explore_column(df, "KOSPI200_Ratio_60", "VKOSPI")


df["Month"] = df.index.month
df.head()

make_violin_plot("Month")

df = pd.get_dummies(df, columns=["Month"])
df.head()

X = df.drop("VKOSPI", axis=1)
y = df["VKOSPI"]

train_predict(X, y, "Après one hot encoding sur les mois", metric=RMSE, cv=10)

In [ ]:
Add features: technical indicators
RSI, NATR, ADX, NORM_MIDDLE

Explorer corrélation entre les données


In [ ]:
n_estimators_grid = np.linspace(50, 200, 10, dtype=int)
score_result = []
for n_estimators in n_estimators_grid:
    scores = train_predict(
        X, y, "Random Forest", metric=RMSE, cv=10, n_estimators=n_estimators
    )
    score_result.append(np.mean(scores))

plt.figure(figsize=(15, 8))
plt.plot(n_estimators_grid, score_result, "o-")
plt.xlabel(r"Pénalisation $\lambda$ pour la régression Ridge")
plt.ylabel("Performance (RMSE)")
plt.title(r"Performance de la régression Ridge en fonction de la pénalisation $L_2$")
plt.show()

In [ ]:
## Supprimer column KOSPI200 + ajouter columns de returns sur différent horizon

Nỗi sợ của mọi người ko bị ảnh hưởng bởi việc giá chính xác của ngày hôm đấy là bao nhiêu, mà là giá của ngày hôm đấy so với những ngày trước đấy
horizon = [2, 5, 30, 60]
Thí dụ khi mua stock, mn sẽ muốn mua khi giá đã tăng/giảm so với những ngày trước, trend đang tăng, giảm
df["Returns"] = df["KOSPI200"].pct_change()


def count_positive_returns(returns):
    return (returns > 0).sum()


new_predictors = []
for h in horizon:
    rolling_mean = df["KOSPI200"].rolling(h).mean()
    ratio_column = f"KOSPI200_Ratio_{h}"
    df[ratio_column] = df["KOSPI200"] / rolling_mean

    df["KOSPI200"].shift(h)
    trend_column = f"KOSPI200_Trend_{h}"
    df[trend_column] = (
        df["Returns"].shift(1).rolling(h).apply(count_positive_returns, raw=True)
    )

df.dropna(inplace=True)
df.drop(columns=["Returns"], inplace=True)
df.drop(columns=["KOSPI200"], inplace=True)
df.head(50)

X = df.drop("VKOSPI", axis=1)
y = df["VKOSPI"]

train_predict(X, y, "Après ajouter des nouvelles variables", metric=RMSE, cv=10)

Them cot VKOSPI cua ngay hom qua


In [ ]:
added_columns = [col for col in df.columns.tolist() if col not in original_columns]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assume `X` is your feature matrix
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[predictors])

# Initialize PCA to retain 95% of explained variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Print explained variance ratio for each principal component
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Number of components:", pca.n_components_)

train_predict(
    pd.DataFrame(X_pca),
    df["VKOSPI"],
    "Après reduction de dimension",
    metric=RMSE,
    cv=10,
)

In [ ]:
ps:Dans le marché des options KOSPI200, les options expirent généralement une fois par mois, la date d'expiration étant fixée au deuxième jeudi de chaque mois. Par conséquent, dans les données, la variable Day_till_expiration représente le nombre de jours restants avant la prochaine date d'expiration.
Lorsque les options arrivent à expiration, tous les contrats ouverts non réglés sont automatiquement liquidés ou livrés. Ensuite, un nouveau contrat d'options est généré, avec une date d'expiration fixée au deuxième jeudi du mois suivant

In [ ]:
## Analyse des Corrélations
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 计算相关性矩阵
correlation_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix, annot=True, fmt=".2f", square=True, cbar_kws={"shrink": 0.8}
)
plt.title("Correlation Matrix Heatmap")
plt.show()


high_corr_pairs = correlation_matrix.unstack().sort_values(ascending=False)
high_corr_pairs = high_corr_pairs[
    (high_corr_pairs.abs() > 0.9) & (high_corr_pairs < 1.0)
]

print("Highly correlated pairs (|correlation| > 0.9):")
print(high_corr_pairs)

- La variable **Day_of_a_week** ainsi que **Day_till_expiration** ont un coefficient de corrélation très faible avec **VKOSPI** et peuvent donc être supprimées.
- **Indiv_Future_Netbuying_Quantity** et **For_Future_Netbuying_Quantity** présentent un coefficient de corrélation absolu relativement élevé de **-0,68** entre elles. Cependant, leur corrélation avec la colonne cible **VKOSPI** est très faible (**0,01**), ce qui permet de considérer leur suppression.
